In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from pathlib import Path
import numpy as np
import pandas as pd

from sklearn.utils import resample

from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score

from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.model_selection import cross_val_score

import lightgbm as lgb

import time

In [3]:
## Folder path
folder_path = Path.cwd().joinpath('drive', 'My Drive', 'Colab Notebooks', 'customer_churn_prediction', 'data')

In [4]:
## Read training and validation data
train_val_data = pd.read_csv(folder_path.joinpath('train_val_data.csv'))
train_val_data.head()

,msno,is_churn,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,city,bd,gender,registered_via,registration_init_time,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
0,YSs/vsH+AL1WRvlkoLDGFT9wWihtQnqQZuKhqdcybm0=,0,40.0,30.0,149.0,149.0,1.0,2017-03-22,2017-04-22,0.0,18.0,0.0,NaN,9.0,2012-04-18,116.0,26.0,25.0,37.0,3541.0,2912.0,866766.109
1,WxkFqbfXFSQgzlLld/tgOEFA9oGpf7JmPgsaMryCZWg=,0,41.0,30.0,149.0,149.0,1.0,2017-03-05,2017-04-05,0.0,4.0,40.0,female,7.0,2013-09-06,16.0,3.0,3.0,1.0,20.0,33.0,6766.055
2,58GesZdrmqPIpKnmdzPnUpw07joB4w8ayPNVfhMPPY4=,0,40.0,30.0,149.0,149.0,1.0,2017-03-24,2017-04-24,0.0,10.0,35.0,male,9.0,2014-12-24,127.0,45.0,29.0,22.0,1305.0,1060.0,318577.925
3,4LqpGjJ/MUOT3a0WEcSUI6xEykQACYUX0pCPm0xgVsg=,1,35.0,7.0,0.0,0.0,0.0,2017-02-20,2017-09-29,0.0,1.0,0.0,NaN,9.0,2014-12-04,45.0,5.0,7.0,11.0,528.0,560.0,137037.344
4,limE8R97wWE+cNSl8CXf0CG/wmI4m1WaZQYPxcjz184=,0,41.0,30.0,149.0,149.0,1.0,2017-03-27,2017-04-27,0.0,1.0,0.0,NaN,7.0,2011-12-17,98.0,25.0,14.0,10.0,484.0,399.0,125289.625


In [5]:
## Read test data
test_data = pd.read_csv(folder_path.joinpath('test_data.csv'))
test_data.head()

,msno,is_churn,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,city,bd,gender,registered_via,registration_init_time,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
0,/lUtWg8ExxX7WttR/T0UEV5ZmHSjlI+2kEbHdOLQu4o=,0,41.0,30.0,100.0,100.0,1.0,2017-03-20,2017-04-20,0.0,6.0,26.0,female,7.0,2017-01-21,76.0,14.0,12.0,3.0,224.0,109.0,51682.863
1,ZKuhr2ZHSE6jAd25Mv1DC/f7q2oZUV9NpxF//1KAC/E=,0,41.0,30.0,99.0,99.0,1.0,2017-03-31,2017-04-30,0.0,1.0,0.0,NaN,7.0,2015-10-28,43.0,20.0,18.0,16.0,94.0,180.0,33348.512
2,Kea1xZZjKRAB3aaYIB1BSM580q2pqxxW/dYId7fEcqU=,0,39.0,30.0,149.0,149.0,1.0,2017-03-31,2017-05-05,0.0,15.0,50.0,male,9.0,2012-03-03,72.0,24.0,21.0,36.0,292.0,356.0,85705.528
3,IesOUKSq3onwcJe/MXEBAEw0U/8+qhFa4GQPAxm8EAo=,0,41.0,30.0,99.0,99.0,1.0,2017-03-24,2017-04-24,0.0,1.0,0.0,NaN,7.0,2016-04-25,50.0,20.0,11.0,8.0,666.0,608.0,163959.090
4,HWAZShjWZK98M68I5CJq03m7Mgiq/vmN53/GkXGi4+E=,0,41.0,30.0,99.0,99.0,1.0,2017-03-31,2017-04-30,0.0,1.0,0.0,NaN,7.0,2016-01-31,46.0,6.0,4.0,6.0,135.0,132.0,34568.674


## Feature Engineering


### Add a new column for price discount

In [6]:
train_val_data['price_discount'] = train_val_data['plan_list_price'] - train_val_data['actual_amount_paid']

In [7]:
sum(train_val_data['price_discount'] < 0) ## There are 3 datapoints for which price discount is negative. Adjust those impure datapoints

3

In [8]:
train_val_data.loc[train_val_data['price_discount'] < 0, :]

,msno,is_churn,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,city,bd,gender,registered_via,registration_init_time,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs,price_discount
259910,NcZX/RK3ZC8tdatMRed2g5T21FoeahEOFp5zfyX2xl0=,1,41.0,0.0,0.0,149.0,1.0,2015-05-13,2017-04-17,0.0,1.0,0.0,NaN,7.0,2011-06-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-149.0
742078,45UpnCjLzdXmZx6QtGiiylUb6bdaBzwg2UXwukwotrw=,1,38.0,30.0,126.0,127.0,0.0,2015-12-09,2017-04-15,0.0,13.0,25.0,female,9.0,2014-07-12,111.0,24.0,13.0,17.0,412.0,520.0,116931.92,-1.0
758911,IC9SswMka7S1+iCPxMjxb9S/cwR1LWdPotaDhrqKxXo=,1,38.0,30.0,126.0,127.0,0.0,2015-12-25,2020-07-19,0.0,5.0,34.0,male,9.0,2010-05-13,31.0,63.0,28.0,12.0,477.0,580.0,116087.23,-1.0


In [9]:
train_val_data.loc[[742078,758911], 'plan_list_price'] = 127.0

In [10]:
train_val_data.loc[259910, 'plan_list_price'] = 149.0

In [11]:
## Recompute price discount
train_val_data['price_discount'] = train_val_data['plan_list_price'] - train_val_data['actual_amount_paid']

In [12]:
## Split train_val data into train and val data
train_df, val_df = train_test_split(train_val_data, test_size=0.15, random_state=10)

In [13]:
train_df.head()

,msno,is_churn,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,city,bd,gender,registered_via,registration_init_time,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs,price_discount
66983,5b20roNDwa3p3z0Y3aWtYMuG0dGRB0tDAIwiYHm38mw=,0,41.0,30.0,149.0,149.0,1.0,2017-03-06,2017-04-06,0.0,1.0,0.0,NaN,7.0,2016-04-06,367.0,57.0,30.0,26.0,772.0,1082.0,196341.137,0.0
242418,O+PqZOEN+6Fq5wr+dpntGdnzaSpnUPUIj3el0IAylYY=,0,41.0,30.0,99.0,99.0,1.0,2017-03-10,2017-04-10,0.0,1.0,0.0,NaN,7.0,2014-01-27,263.0,32.0,33.0,44.0,713.0,954.0,194550.134,0.0
455493,lNmJE7vrHFf9/Zp2iGgayiNJMt9Wi17i7EHTBenArIY=,0,39.0,30.0,149.0,149.0,1.0,2017-03-31,2017-05-15,0.0,13.0,27.0,female,9.0,2013-09-14,690.0,136.0,93.0,92.0,2047.0,1829.0,570913.226,0.0
866243,Zf62RXGDnyehoy837HpuoOQy9WdJFPiltWqBcZP+2F0=,0,41.0,30.0,99.0,99.0,1.0,2017-03-14,2017-04-14,0.0,1.0,0.0,NaN,7.0,2016-05-15,35.0,17.0,4.0,13.0,310.0,190.0,92633.412,0.0
83612,rlyej0t+xs1y8MIjClijpXSatK8QOzBI0kok5HO7FtA=,0,40.0,30.0,149.0,149.0,1.0,2017-03-12,2017-04-11,0.0,5.0,57.0,male,9.0,2007-10-07,16.0,4.0,3.0,4.0,593.0,423.0,129521.047,0.0


In [14]:
## Numeric features imputer
numeric_feature_imputer = SimpleImputer(strategy='median')
numeric_columns = ['plan_list_price', 'actual_amount_paid', 'bd', 'num_25', 'num_50', 'num_75', 'num_985', 'num_100', 'num_unq', 'total_secs', 'price_discount']

## Categorical features imputer
categorical_feature_imputer = SimpleImputer(strategy='most_frequent')
categorical_columns = ['payment_method_id', 'payment_plan_days', 'is_auto_renew', 'is_cancel', 'city', 'registered_via']

## Constant imputer
constant_imputer = SimpleImputer(strategy='constant', fill_value='not_specified')
constant_impute_column = ['gender']
constant_imputer_pipeline = Pipeline([('imputer', constant_imputer), ('label_encoder', OrdinalEncoder())])

## Combine all the imputation transformers with Column Transformer
column_transformer = ColumnTransformer(transformers=[('numeric_imputer', numeric_feature_imputer, numeric_columns),
                                                     ('categorical_imputer', categorical_feature_imputer, categorical_columns),
                                                     ('constant_imputer', constant_imputer_pipeline, constant_impute_column)])

## Final pipeline with scaling
pipeline = Pipeline([('imputation_step', column_transformer)])

In [17]:
pipeline.fit(train_df)

Pipeline(memory=None,
         steps=[('imputation_step',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('numeric_imputer',
                                                  SimpleImputer(add_indicator=False,
                                                                copy=True,
                                                                fill_value=None,
                                                                missing_values=nan,
                                                                strategy='median',
                                                                verbose=0),
                                                  ['plan_list_price',
                                                   'actual_amount_paid', 'bd',
                                                   'num_

In [18]:
## Pre-process train data
x_train_tot = pipeline.transform(train_df)
y_train_tot = train_df.is_churn.values

In [19]:
## Preprocess validation data
x_val = pipeline.transform(val_df)
y_val = val_df.is_churn.values

## Downsample data

In [20]:
## Downsample data with class 0 
x_train_0 = resample(x_train_tot[y_train_tot == 0], replace=False, n_samples=int(len(x_train_tot[y_train_tot==0])*0.30), random_state=10)

x_train = np.vstack((x_train_0, x_train_tot[y_train_tot == 1]))
y_train = np.hstack((np.zeros(len(x_train_0), dtype=int), np.ones(len(x_train_tot[y_train_tot == 1]), dtype=int)))

## Light GBM model

### Base model

In [21]:
## Base model Lightgbm classifier
lgb_model = lgb.LGBMClassifier()

start_time = time.time()
lgb_model.fit(x_train, y_train, categorical_feature=[x_train.shape[1]-1])
print(f'Run time: {round(time.time() - start_time, 4)}')

/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [17]
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Run time: 3.7763


In [22]:
## F1 score for training data
f1_score(y_train, lgb_model.predict(x_train))

0.7993965531936655

In [23]:
## F1 score for validation data
f1_score(y_val, lgb_model.predict(x_val))

0.7457362202914539

In [24]:
## CV score for baseline
baseline_cv_score = cross_val_score(lgb_model, x_train, y_train, scoring='f1', cv=5)
print(f"Baseline CV score: {np.mean(baseline_cv_score)}")

Baseline CV score: 0.7985421537228943


In [ ]:
## Randomized Grid search CV
parameters_grid = {'boosting_type':['gbdt', 'goss'],
                   'learning_rate':[0.001, 0.01, 0.1],
                   'max_depth':[3, 5, 10, 50],
                   'n_estimators':[50, 100, 500, 1000],
                   'subsample':[1,0.8],
                   'reg_alpha':[0,0.5],
                   'reg_lambda':[0,0.5],
                   'min_data_in_leaf':[100, 500, 1000],
                   'num_leaves':[6, 24, 500]}

lgb_model = lgb.LGBMClassifier(random_state=7)
lgb_grid_search = RandomizedSearchCV(estimator=lgb_model, n_iter=300, param_distributions=parameters_grid, scoring='f1', cv=5, verbose=3)

start_time = time.time()
lgb_grid_search.fit(x_train, y_train, categorical_feature = [x_train.shape[1]-1])
print(f'Run time: {round(time.time() - start_time, 4)}')

Fitting 5 folds for each of 300 candidates, totalling 1500 fits
[CV] subsample=1, reg_lambda=0, reg_alpha=0.5, num_leaves=6, n_estimators=1000, min_data_in_leaf=1000, max_depth=5, learning_rate=0.01, boosting_type=gbdt 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [17]
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[CV]  subsample=1, reg_lambda=0, reg_alpha=0.5, num_leaves=6, n_estimators=1000, min_data_in_leaf=1000, max_depth=5, learning_rate=0.01, boosting_type=gbdt, score=0.801, total=  22.1s
[CV] subsample=1, reg_lambda=0, reg_alpha=0.5, num_leaves=6, n_estimators=1000, min_data_in_leaf=1000, max_depth=5, learning_rate=0.01, boosting_type=gbdt 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   22.1s remaining:    0.0s
/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [17]
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[CV]  subsample=1, reg_lambda=0, reg_alpha=0.5, num_leaves=6, n_estimators=1000, min_data_in_leaf=1000, max_depth=5, learning_rate=0.01, boosting_type=gbdt, score=0.800, total=  21.6s
[CV] subsample=1, reg_lambda=0, reg_alpha=0.5, num_leaves=6, n_estimators=1000, min_data_in_leaf=1000, max_depth=5, learning_rate=0.01, boosting_type=gbdt 


...

[CV]  subsample=1, reg_lambda=0, reg_alpha=0.5, num_leaves=6, n_estimators=100, min_data_in_leaf=100, max_depth=10, learning_rate=0.1, boosting_type=goss, score=0.791, total=   2.9s
[CV] subsample=1, reg_lambda=0, reg_alpha=0.5, num_leaves=6, n_estimators=100, min_data_in_leaf=100, max_depth=10, learning_rate=0.1, boosting_type=goss 


/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [17]
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[CV]  subsample=1, reg_lambda=0, reg_alpha=0.5, num_leaves=6, n_estimators=100, min_data_in_leaf=100, max_depth=10, learning_rate=0.1, boosting_type=goss, score=0.798, total=   2.8s


[Parallel(n_jobs=1)]: Done 1500 out of 1500 | elapsed: 304.1min finished
/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [17]
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Run time: 18251.3556


In [ ]:
## Save best model
import joblib
model_name = 'LightGBM Classifier'
            
Path.cwd().joinpath('saved_model').mkdir(parents=True, exist_ok=True)
save_loc = Path.cwd().joinpath('saved_model', model_name+'.pkl')

try:
    joblib.dump(lgb_grid_search.best_estimator_, save_loc)
except:
    raise IOError("Could not save the model. Check model name and file path")
else:
    print("Model saved successfully!")

Model saved successfully!


In [ ]:
## Performance on validation data
y_pred = lgb_grid_search.best_estimator_.predict(x_val)
print(f"F1 score : {f1_score(y_val, y_pred)}")

F1 score : 0.7450025274573778


In [ ]:
lgb_grid_search.best_score_

0.7985623741714385

In [ ]:
lgb_grid_search.best_params_

{'boosting_type': 'goss',
 'learning_rate': 0.1,
 'max_depth': 10,
 'min_data_in_leaf': 100,
 'n_estimators': 50,
 'num_leaves': 500,
 'reg_alpha': 0.5,
 'reg_lambda': 0.5,
 'subsample': 1}

In [ ]:
## Do a quick grid search around chosen parameters
parameters_grid_final = {'boosting_type':['goss'],
                   'learning_rate':[0.08, 0.09, 0.1],
                   'max_depth':[8, 10, 12],
                   'n_estimators':[50, 60],
                   'subsample':[1],
                   'reg_alpha':[0.5],
                   'reg_lambda':[0.5],
                   'min_data_in_leaf':[100],
                   'num_leaves':[400, 500]}

lgb_model = lgb.LGBMClassifier(random_state=7)
lgb_grid_search_final = GridSearchCV(estimator=lgb_model, param_grid=parameters_grid_final, scoring='f1', cv=5, verbose=3)

start_time = time.time()
lgb_grid_search_final.fit(x_train, y_train, categorical_feature = [x_train.shape[1]-1])
print(f'Run time: {round(time.time() - start_time, 4)}')

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[CV] boosting_type=goss, learning_rate=0.08, max_depth=8, min_data_in_leaf=100, n_estimators=50, num_leaves=400, reg_alpha=0.5, reg_lambda=0.5, subsample=1 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [17]
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[CV]  boosting_type=goss, learning_rate=0.08, max_depth=8, min_data_in_leaf=100, n_estimators=50, num_leaves=400, reg_alpha=0.5, reg_lambda=0.5, subsample=1, score=0.803, total=   2.7s
[CV] boosting_type=goss, learning_rate=0.08, max_depth=8, min_data_in_leaf=100, n_estimators=50, num_leaves=400, reg_alpha=0.5, reg_lambda=0.5, subsample=1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.7s remaining:    0.0s
/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [17]
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[CV]  boosting_type=goss, learning_rate=0.08, max_depth=8, min_data_in_leaf=100, n_estimators=50, num_leaves=400, reg_alpha=0.5, reg_lambda=0.5, subsample=1, score=0.803, total=   2.5s
[CV] boosting_type=goss, learning_rate=0.08, max_depth=8, min_data_in_leaf=100, n_estimators=50, num_leaves=400, reg_alpha=0.5, reg_lambda=0.5, subsample=1 


...

[CV]  boosting_type=goss, learning_rate=0.1, max_depth=12, min_data_in_leaf=100, n_estimators=60, num_leaves=500, reg_alpha=0.5, reg_lambda=0.5, subsample=1, score=0.794, total=   4.4s
[CV] boosting_type=goss, learning_rate=0.1, max_depth=12, min_data_in_leaf=100, n_estimators=60, num_leaves=500, reg_alpha=0.5, reg_lambda=0.5, subsample=1 


/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [17]
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[CV]  boosting_type=goss, learning_rate=0.1, max_depth=12, min_data_in_leaf=100, n_estimators=60, num_leaves=500, reg_alpha=0.5, reg_lambda=0.5, subsample=1, score=0.800, total=   4.0s


[Parallel(n_jobs=1)]: Done 180 out of 180 | elapsed:  9.8min finished
/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [17]
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Run time: 592.7403


In [49]:
lgb_grid_search_final.best_score_

0.798718974954444

In [50]:
## Performance on validation data
y_pred = lgb_grid_search_final.best_estimator_.predict(x_val)
print(f"F1 score : {f1_score(y_val, y_pred)}")

F1 score : 0.744600680084551


Although the change is marginal, it looks like the data has started to marginally overfit. Hence, sticking to the best model from Randomized Grid Search

In [56]:
from sklearn.metrics import log_loss

In [60]:
log_loss(y_val, lgb_grid_search_final.best_estimator_.predict_proba(x_val))

0.1637570560297849